Note: applying window function was attampted, but not applied to the final models.

Main contributor: Shengbo Zhang

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import findspark  # Get rid of this in DataBricks
findspark.init()  # Get rid of this in DataBricks
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.ml.feature import PCA
from pyspark.ml.classification import GBTClassifier


In [ ]:
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  # Jjoin them together

In [ ]:
gene_sub_df = df2.select(*gene_feature_names)
gene_sub_df_T = spark.createDataFrame(gene_sub_df.toPandas().T)

w = Window.rowsBetween(-10,0)


gene_sub_df_T_roll = gene_sub_df_T.select(
    '*', 
    *( F.avg(i).over(w).alias(i + '_roll') for i in gene_sub_df_T.columns)
).drop(*gene_sub_df_T.columns)  # apply rolling average transformation for each sample
gene_rolled = spark.createDataFrame(gene_sub_df_T_roll.toPandas().T)

gene_rolled = gene_rolled.select([col(c).alias('g_' + c + '_rolled') for c in gene_rolled.columns])\
                                .drop(*gene_rolled.columns)  # just renameing the columns... that's all


window = Window.orderBy(F.col('monotonically_increasing_id'))
gene_rolled = gene_rolled.withColumn("monotonically_increasing_id", F.monotonically_increasing_id())\
                        .withColumn('row_number2', F.row_number().over(window))\
                        .drop('monotonically_increasing_id')

gene_rolled.write.parquet('gene_rolled.parquet')